# Grounding LLM statistics facts using Retrieval Interleaved Generation (RIG)

In this notebook, we share promising, early research advancements that tackle the challenge of provenance around real-world statistical data. This notebook connects to DataGemma, the first [open model](https://huggingface.co/gg-hf/datagemma-rig-27b-it) designed to connect large language models with the extensive, real-world data housed within Google's [Data Commons](https://datacommons.org).

This novel approach fine-tunes [Gemma 2](https://ai.google.dev/gemma/docs) to recognize when it needs to replace a generated number with more accurate information from Data Commons. Think of it as the model double-checking its work against a trusted source. More technical details of this approach can be found in this [paper](https://datacommons.org/link/DataGemmaPaper).

This demo is based on a finetuned Gemma2 27B model.

Please read [Gemma Terms of Use](https://ai.google.dev/gemma/terms).

***Disclaimer:***

*You're accessing a very early version of DataGemma. It is meant for trusted tester use (primarily for academic and research use) and not yet ready for commercial or general public use. This version was trained on a very small corpus of examples and may exhibit unintended, and at times controversial or inflammatory behavior. Please anticipate errors and limitations as we actively develop this large language model interface.*

*Your feedback and evaluations are critical to refining DataGemma's performance and will directly contribute to its training process. Known limitations are detailed in the paper, and we encourage you to consult it for a comprehensive understanding of DataGemma's current capabilities.*

## Step 0: Setup

To run this colab, you will need to use the A100 GPU and High-RAM runtime in Colab. With this runtime configuration, the total runtime of this notebook would take about 20 minutes.

You also need authentication for model and data access:

*   **Hugging Face Token**. To obtain the token, login to your Hugging Face account [token settings](https://huggingface.co/settings/tokens) to create a new token. Copy this token and store it on the colab notebook `Secrets` section with Name `HF_TOKEN`.

*   **Data Commons API Key**. Register for an API key from Data Commons [API key portal](https://apikeys.datacommons.org). Once you get the API key, store it on the colab notebook `Secrets` section with Name `DC_API_KEY`.

Toggle the "Notebook access" button to enable the secrets.

Then install the required libraries.

In [1]:
!pip install -q git+https://github.com/datacommonsorg/llm-tools
!pip install -q bitsandbytes accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.6 MB/s eta 0:00:00


## Step 1: Load the model

This section loads the finetuned Gemma2 27B model from Huggingface and creates a transformer model wrapper than can be used in the Retrieval Interleaved Generation (RIG) workflow. More technical details of this approach can be found in the [paper](https://datacommons.org/link/DataGemmaPaper).

In [10]:
import torch

import data_gemma as dg

from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Initialize Data Commons API client
DC_API_KEY = userdata.get('DC_API_KEY')
dc = dg.DataCommons(api_key=DC_API_KEY)


# Get finetuned Gemma2 model from HuggingFace
HF_TOKEN = userdata.get('HF_TOKEN')

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = 'gg-hf/data-gemma-rig-27b-it' # TODO(update)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
datagemma_model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=nf4_config,
                                             torch_dtype=torch.bfloat16,
                                             token=HF_TOKEN)

# Build the LLM Model stub to use in RIG flow
datagemma_model_wrapper = dg.HFBasic(datagemma_model, tokenizer)

## Step 2: Pick or enter a query for RIG

You can selected a query or enter your own query to test RIG.


In [3]:

#@title Pick a query from a sample list{ run: "auto" }
QUERY = "What progress has Pakistan made against health goals?" #@param ["What percentage of the Indian population lives in slums?","In which countries are more women getting college degrees than men?","What is the percentage of the financial sector in GDP for different countries like the United States and China, based on the latest data?","Which New Jersey Towns have the best commute times for workers?","Does India have more people living in the urban areas or rural areas?  How does that vary by states?  Are the districts with the most urban population also located in the states with the most urban population?","What are some interesting trends in Sunnyvale spanning gender, age, race, immigration, health conditions, economic conditions, crime and education?","In the US states with the highest foreign language speakers, how does the unemployment rate compare to the national average?","There is probably a delay between a high percentage of females attending school and females being in elected positions in the government. Can you show me what that delay looks like in different African countries?","Which US counties share a very similar demographic composition to the US overall in terms of gender, age and racial breakdown?","Compare Cambridge, MA and Palo Alto, CA in terms of demographics, education, and economy stats.","How does the size of household compare across counties in Utah vs. California?  Does it change between owned vs. rental properties?","When comparing median ages across different racial groups in various cities in California, what differences emerge?","Based on the distribution of foreign language speakers compare the diversity of people in: NYC, Seattle, Austin, Chicago and Tampa","Are there significant differences in the prevalence of various types of disabilities (such as vision, hearing, mobility, cognitive) between Dallas and Houston?","It might be expected that countries producing the most food waste have a lower prevalence of undernourishment. Does the data support this?","Are there countries in the world where the forest area has actually increased?","What progress has Pakistan made against health goals?","Does an increase in female participation in education result in a higher number of women holding political office?","Which countries have the highest life expectancy?","Which countries have the lowest poverty rates?","Which countries have the highest GDP?","Has the use of renewables increased globally?","Has the average lifespan increased globally?"]


In [4]:
#@title Use your own query (Please see disclaimer at the top)
QUERY = 'In the US states with the highest foreign language speakers, how does the unemployment rate compare to the national average?' #@param {type:"string"}

## Step 3: Run RIG and Print Output


In [11]:
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


ans = dg.RIGFlow(llm=datagemma_model_wrapper, data_fetcher=dc, verbose=False).query(query=QUERY)
Markdown(textwrap.indent(ans.answer(), '> ', predicate=lambda _: True))


display_chat(QUERY, ans.answer())

... calling HF Pipeline API "In the US states with the highest foreign language..."
... calling DC with "what was the unemployment rate for foreign-born individuals in the US in 2021?"
... calling DC with "what percentage of agricultural workers speak spanish at home in the US?"
... calling DC with "what percentage of agricultural workers speak english at home in the US?"
... calling DC with "what percentage of construction workers speak spanish at home in the US?"
... calling DC with "what percentage of construction workers speak english at home in the US?"
... calling DC with "what percentage of healthcare workers speak spanish at home in the US?"
... calling DC with "what percentage of healthcare workers speak english at home in the US?"
... calling DC with "what percentage of social assistance workers speak spanish at home in the US?"
... calling DC with "what percentage of social assistance workers speak english at home in the US?"


<font size='+1' color='brown'>🙋‍♂️<blockquote>In the US states with the highest foreign language speakers, how does the unemployment rate compare to the national average?</blockquote></font><font size='+1' color='teal'>🤖

> 
> 
> The US Bureau of Labor Statistics doesn't publish specific unemployment rates for individuals who speak a high percentage of their working hours in a foreign language.
> 
> However,
> 
> * **They do publish statistics on foreign-born individuals in the workforce.** According to the BLS, foreign-born individuals have a lower unemployment rate than native-born individuals. As of 2021, the unemployment rate for foreign-born individuals was **[__DC__#1(6922847 [1]* || 3.1%)]**.
> * **Industries with higher foreign language speaker concentrations:**
> 
>     * **Agriculture:** [__DC__#2(|| 53.8%)] of agricultural workers speak spanish at home, compared to [__DC__#3(|| 46.1%)] who speak English.
>     * **Construction:** [__DC__#4(|| 45.7%)] of construction workers speak spanish at home, compared to [__DC__#5(|| 38.8%)] who speak English.
>     * **Healthcare:** [__DC__#6(|| 38.5%)] of healthcare workers speak spanish at home, compared to [__DC__#7(|| 36.6%)] who speak English.
>     * **Social assistance:** [__DC__#8(|| 35.9%)] of social assistance workers speak spanish at home, compared to [__DC__#9(|| 32.6%)] who speak English.
> 
> * **Location:** Unemployment rates tend to be lower in states with larger Hispanic or Asian populations.
> 
> **Important considerations:**
> 
> * **Data limitations:** Unemployment statistics often don't capture the full extent of language barriers in the workforce. Many individuals who speak a foreign language may be employed, but face challenges such as communication, training, and access to opportunities.
> * **Industry variations:** Unemployment rates can vary significantly across industries. STEM fields, for example, tend to have lower unemployment rates than humanities fields.
> * **Regional differences:** Unemployment rates can also vary regionally within a state, depending on factors like geographic location, company size, and individual qualifications.
> 
> **Where to find more information:**
> 
> * **U.S. Bureau of Labor Statistics:** https://www.bls.gov/
> * **Migration Policy Institute:** https://www.migrationpolicy.org/
> * **American Immigration Council:** https://www.ai-council.org/
> 
> It's important to remember that these are just overarching statistics, and unemployment rates can fluctuate within specific industries and regions. 
> 
> 
> #### FOOTNOTES ####
> [1] - Per census.gov, value was 6922847 in 2021. See more at https://datacommons.org/explore#q=what%20was%20the%20unemployment%20rate%20for%20foreign-born%20individuals%20in%20the%20US%20in%202021%3F&mode=toolformer_rig
</font>

# More Information on Retrieval Interleaved Generation (RIG)

Retrieval Interleaved Generation (RIG): This novel approach fine-tunes Gemma 2 to recognize when it needs to replace a generated number with more accurate information from Data Commons. Think of it as the model double-checking its work against a trusted source.

Here's how RIG works:
1. User Query: A user submits a query to the LLM.
2. Initial Response & Data Commons Query: The DataGemma model (based on the Gemma 2 27 billion parameter (27B) model and fully fine-tuned for this RIG task) generates a response, which includes a natural language query for Data Commons' existing natural language interface,  specifically designed to retrieve relevant data.
3. Data Retrieval & Correction: Data Commons is queried, and the data are retrieved. These data, along with source information and a link, are then used to replace potentially inaccurate numbers in the initial response.
4. Final Response with Source Link: The final response is presented to the user, including a link to the source data and metadata in Data Commons for transparency and verification.

In the above example, notice the questions being asked of Data Commons (eg "what was the life expectancy in Pakistan in 2000?") which is being used to compare the initial LLM response in `[__DC__#1(62.102 yr [1] || 61.8 years)]`. `61.8 years` is the  value generated by Gemma2 27B. DataGemma is trained to query Data Commons with "what was the life expectancy in Pakistan in 2000?". Statistics from the World Bank along with citations to the initial source are returned by Data Commons and replaced in the final response.  